In [1]:
from selenium.webdriver.support.ui import WebDriverWait
from tqdm.notebook import tqdm_notebook,tqdm
from selenium.webdriver.common.by import By
from IPython.display import display
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pytesseract
import json
import time
import sys
import re
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [3]:
url = 'https://www.zaubacorp.com/user/login?destination=node'
driver = webdriver.Chrome('chromedriver',options=chrome_options)
driver.get(url)

In [4]:
element_ = driver.page_source
# print(element_)

In [5]:
def login(cuser_text, pass_text, captcha_text):
    '''
    Login and access page
    '''
    cuser = driver.find_element(By.ID, "edit-name")
    cpass = driver.find_element(By.ID, "edit-pass")
    cpt = driver.find_element(By.ID, "edit-captcha-response")
    cuser.send_keys(cuser_text)
    cpass.send_keys(pass_text)
    cpt.send_keys(captcha_text)
    driver.find_element(By.ID, "edit-submit").click()

In [6]:
t = driver.find_element(By.XPATH,"//div[@class='form-type-textfield form-item-captcha-response form-item form-group']")
variables = ''.join(list(filter(lambda x:len(x)!=0,re.split(r'[Math question *]*',t.text)))[:-1])
ans = eval(variables)
login('prince404','PRINCE@123',str(ans))

In [7]:
print(driver.current_url)
# Check this wait
WebDriverWait(driver=driver, timeout=10).until(
    lambda x: x.execute_script('return document.readyState') == 'complete'
)
print(driver.current_url)
time.sleep(7)
print(driver.current_url)

https://www.zaubacorp.com/user/login?destination=node
https://www.zaubacorp.com/user/login?destination=node
https://www.zaubacorp.com/node


In [8]:
def display_all(data):
    for i in range(len(data)):
        print(i)
        display(data[i])

In [9]:
def load_page(link):
    driver.get(link)
    WebDriverWait(driver=driver, timeout=10).until(
    lambda x: x.execute_script('return document.readyState') == 'complete'
    )
    print(driver.current_url)
    time.sleep(3)

def basic_details(data,d_company):
    temp = np.array(data)[0]
    for i in range(len(temp)):
        if((temp[i][1]!='-') and (temp[i][1]!='') and (str(temp[i][1])!='nan')):
            d_company[temp[i][0]]=temp[i][1]

def pre_data(data,d_company,val):
    temp = np.array(data).tolist()[0]
    if(len(temp)<1 or (len(temp)==1 and 'found' in temp[0][0].split(' '))):
        return
    l = []
    for i in range(len(temp)):
        l.append(temp[i][0])
    d_company[val]=l

def establishments(data,d_company):
    temp = np.array(data).tolist()[0]
    if(len(temp)<1 or (len(temp)==1 and 'found' in (temp[0][0].lower()).split(' '))):
        return
    l = []
    for i in range(len(temp)):
        l.append(temp[i][3])
    d_company['Establishments'] = l

def charges(data,d_company):
    temp = np.array(data).tolist()[0]
    if(len(temp)<1 or (len(temp)==1 and 'found' in str(temp[0][0]).split(' '))):
        return
    l = []
    amount = 0
    for i in range(len(temp)):
        if(temp[i][5]!=float('nan') or str(temp[i][5]).isnumeric()):
            amount+=int(temp[i][5])
        l.append(int(temp[i][0]))
    d_company['Charges IDs'] = l
    d_company['Cahrges/Borrowing Amount'] = amount
    d_company['Number of Charges']=len(temp)

def persecution(data,d_company):
    temp = np.array(data).tolist()[0]
    if(len(temp)<1 or (len(temp)==1 and 'found' in (temp[0][0].lower()).split(' '))):
        return
    # l = []
    # for i in range(len(temp)):
    #     l.append(temp[i][3])
    # d_company['Persecution'] = l
    d_company['Number of Persecutions']=len(temp)

def cur_directors(data,d_company):
    temp = np.array(data[0]).tolist()
    if(len(temp)<1 or (len(temp)==1 and 'not' in (temp[0][0].lower()).split(' '))):
        return
    l = []
    for i in range(len(temp)):
        if(str(temp[i][0]).isdecimal()):
            l.append(temp[i][1])
    d_company['Current Directors'] = l

In [10]:
def fetch_data(link):
    load_page(link)
    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')
    temp_data = soup.find_all('table')
    if(len(temp_data)<5):
        print("Company Not Found!")
        return
    d_company = {}
    basic_details(pd.read_html(str(temp_data[0]),header=None),d_company)
    basic_details(pd.read_html(str(temp_data[3]),header=None),d_company)
    basic_details(pd.read_html(str(temp_data[4]),header=None),d_company)
    basic_details(pd.read_html(str(temp_data[5]),header=None),d_company)
    basic_details(pd.read_html(str(temp_data[6]),header=None),d_company)
    try:
        pre_data(pd.read_html(str(temp_data[1]),header=None),d_company,'Previous Names')
        pre_data(pd.read_html(str(temp_data[2]),header=None),d_company,'Previous CIN')
    except:
        pass
    try :
        establishments(pd.read_html(str(temp_data[-1]),header=None),d_company)
        charges(pd.read_html(str(temp_data[-2]),header=None),d_company)
        persecution(pd.read_html(str(temp_data[-3]),header=None),d_company)
        cur_directors(pd.read_html(str(temp_data[7]),header=None),d_company)
    except:
        pass
    return d_company

In [11]:
with open('../../data/zaubacorp.com/links.json','r+') as jsonfp:
    company_links = json.load(jsonfp)
    
print("Length of Data :",len(company_links))
company_data = {}
# for checking limit = 100 
#NOTE Change or Remove Limit and Count 
limit = 100
count = 0 
with tqdm_notebook(total=len(company_links)) as pbar:
    for i in company_links:
        if(count > limit):
            break
        json_data = fetch_data(company_links[i]['link'])
        company_data[i] = json_data
        pbar.update()
        count+=1

Length of Data : 140280


  0%|          | 0/140280 [00:00<?, ?it/s]

https://www.zaubacorp.com/company/RUDRANGI-WIRE-PRODUCTS-PRIVATE-LIMITED/U31300TG1991PTC013468
https://www.zaubacorp.com/company/RUDRANI-INFRA-PROJECTS-PRIVATE-LIMITED/U45300TG2008PTC059087
https://www.zaubacorp.com/company/RUDRANI-PHARMACEUTICALS-PRIVATE-LIMITED/U24239TG2003PTC042314
https://www.zaubacorp.com/company/RUDRANI-REALTORS-PRIVATE-LIMITED/U45201TG2021PTC151544
https://www.zaubacorp.com/company/RUDRAPRIYA-DISTRIBUTOR-PRIVATE-LIMITED/U51909TG2010PTC109626
https://www.zaubacorp.com/company/RUDRAVEERYA-ENTERPRISES-PRIVATE-LIMITED/U70109TG2021PTC148201
https://www.zaubacorp.com/company/RUDRAVEL-PRIVATE-LIMITED/U52609TG2020PTC141552
https://www.zaubacorp.com/company/RUDRESHWARA-WOMEN-FARMERS-PRODUCER-COMPANY-LIMITED/U01100TG2021PTC155585
https://www.zaubacorp.com/company/RUDRINI-HDPE-PIPES-AND-FITTINGS-PRIVATE-LIMITED/U25209TG2017PTC114351
https://www.zaubacorp.com/company/RUDVIN-CORPORATION-INDIA-PRIVATE-LIMITED/U24239TG2006PTC048783
https://www.zaubacorp.com/company/RUGENT-CONS

In [12]:
file = open("../../data/zaubacorp.com/Companies.json", "w+")
json.dump(company_data, file, indent=4)
file.close()